In [ ]:
%matplotlib inline

import traceback
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic
from IPython.display import display, Audio, Image

# C++ and computer architecture

The low-level code of numerical software must be high-performance.  The industries chose C++ because it can take advantage of everything that a hardware architecture offers while using any level of abstraction.

1. Fundamental data types
   1. Command-line interface for compiler tools
      1. Compiler, linker
      2. Multiple source files, separation of declaration and definition, external libraries
      3. Build multiple binaries and shared objects (dynamically linked libraries)
   2. Integer, signness, pointer, array indexing
   3. Floating-point, rounding, exception handling
   4. Numeric limit
2. Object-oriented programming
   1. Class, encapsulation, accessor, reference type
   2. Constructor and destructor
   3. Polymorphism and RTTI
   4. CRTP
3. Standard template library (STL)
   1. std::vector, its internal and why the buffer address is dangerous
   2. std::array, std::list
   3. std::map, std::set, std::unordered_map, std::unordered_set

## Why C++?

C++ is particularly difficult to learn and use.  If all we need is speed, why not invent an easier language that is equally fast?

Two reasons render the new-language approach incomplete.  Speed is first.  It's just difficult to invent something that is as fast as C++ in all aspects.  C++ is compatible to the language of operating system and accesses assembly easily.  Other languages may outperform C++ in some occasions, but not most.

Another reason is maintenance.  A numerical system usually takes years to develop.  When proven useful, it will be maintained for decades.  Such a long-living system needs a lot of support from the compiler, which is offered by C++.

# Compile and link

In [ ]:
!rm -f helloworld.o
!ls helloworld.o

Compiler is a complex system.  The most common way to use it is to execute the compiler driver from the command-line.  For example, in GCC, the C++ compiler driver is `g++`.  It accepts a lot of command-line arguments.

When running the compiler with the argument `-c`, it takes the source code and output the object file.  The object file contains the machine code, but doesn't include the functions not defined in the source file.

The argument `-o` is used to specify the output file name.

In [ ]:
# Compile source file to object file
!g++ -c helloworld.cpp -o helloworld.o
!file helloworld.o

In [ ]:
# Object file isn't executable yet
!chmod a+x helloworld.o
!./helloworld.o

By dropping the `-c` argument, and supplying the object file as the input, `g++` will link the object file and necessary libraries into the executable.

In [ ]:
# Link into executable; dropping -c
!g++ helloworld.o -o helloworld
!file helloworld

In [ ]:
!./helloworld

## Separate compilation units

#### Header file

`hello.hpp`:

```cpp
#pragma once
void hello();
```

`hello.cpp`:

```cpp
#include <iostream>
#include "hello.hpp"
void hello() { std::cout << "hello with standalone compiling unit" << std::endl; }
```

In [ ]:
# There's no "main()"; cannot link
!g++ hello.cpp -o hello

In [ ]:
# hello.cpp needs to be built as an object file
!g++ -c hello.cpp -o hello.o

### Main program

`hellomain.cpp`:

```cpp
#include "hello.hpp"
int main(int argc, char ** argv) { hello(); }
```

In [ ]:
# hellomain.cpp doesn't have everything it needs; cannot link
!g++ hellomain.cpp -o hellomain

### Link object files

In [ ]:
# hellomain.cpp also needs to be built as an object file
!g++ -c hellomain.cpp -o hellomain.o

In [ ]:
# Link into executable
!g++ hellomain.o hello.o -o hellomain

In [ ]:
# Then it can run
!./hellomain

## Create and use a static library

In [ ]:
!g++ -c hello.cpp -o hello.o
!ar rcs libhello.a hello.o
!g++ hellomain.o -L. -lhello -o hellomain2
!./hellomain2

## Create and use a shared object

In [ ]:
!g++ -c hello.cpp -o hello.o
!g++ -shared hello.o -o libshared_hello.so
!g++ hellomain.o -L. -lshared_hello -o hellomain3
!./hellomain3

### External libraries are included in the same way

In [ ]:
!g++ distance.cpp -o distance -lcblas
!./distance

# C++ integer types

Width of the fundamental types is specified by the standard to assist writing portable code.

* Boolean types: `bool`
* Integer (signed) types: `short` (16+ bits), `int` (16+ bits), `long` (32+ bits), `long long` (64+ bits)
  * Unsigned integer: `unsigned short` (16+ bits), `unsigned int` (16+ bits), `unsigned long` (32+ bits), `unsigned long long` (64+ bits)
* Character type: `char`, `unsigned char` (8 bits)

In [ ]:
!g++ types.cpp -o types
!./types

## Fixed-width integer types

The C++ standard library provides the fixed-width (bit) integer types which are the same as C in the header file `<cstdint>`:

* Signed integer: `int8_t`, `int16_t`, `int32_t`, `int64_t`.
* Unsigned integer: `uint8_t`, `uint16_t`, `uint32_t`, `uint64_t`.

Fixed width matters for numerical code more than hardware architecture does.  It's easier for numerical code to break by changed width of indexing integer than by changed addressing space.

`cstdint.cpp`:

```cpp
#include <iostream>
#include <cstdint>
int main(int argc, char ** argv)
{
    std::cout << "unit is byte" << std::endl;
    std::cout << "sizeof(int8_t): " << sizeof(int8_t) << std::endl;
    std::cout << "sizeof(uint8_t): " << sizeof(uint8_t) << std::endl;
    std::cout << "sizeof(int16_t): " << sizeof(int16_t) << std::endl;
    std::cout << "sizeof(uint16_t): " << sizeof(uint16_t) << std::endl;
    std::cout << "sizeof(int32_t): " << sizeof(int32_t) << std::endl;
    std::cout << "sizeof(uint32_t): " << sizeof(uint32_t) << std::endl;
    std::cout << "sizeof(int64_t): " << sizeof(int64_t) << std::endl;
    std::cout << "sizeof(uint64_t): " << sizeof(uint64_t) << std::endl;
    return 0;
}
```

In [ ]:
!g++ cstdint.cpp -o cstdint
!./cstdint

## Signness

Care should be taken when signed and unsigned integers are both used in code.  Comparison result between signed and unsigned integers is sometimes surprising.

The common wisdom advises to not mixing signed and unsigned integer, but in numerical code negative indices are commonplace.  Be especially careful about the sign.

`signness.cpp`:
    
```cpp
#include <iostream>
#include <cstdint>
int main(int, char **)
{
    long sint = -1;
    unsigned long uint = 1;
    std::cout << "sint: " << sint << std::endl;
    std::cout << "uint: " << uint << std::endl;
    if (sint > uint) { std::cout << "sint > uint, although it can't be" << std::endl; }
    return 0;
}
```

In [ ]:
!g++ signness.cpp -o signness
!./signness

In [ ]:
# Ask the compiler to check for incorrect sign.
!g++ signness.cpp -Wsign-compare -Werror -o signness

## Pointer and array indexing

`arrays.cpp`:

```cpp
#include <iostream>
#include <cstdint>
int main(int, char **)
{
    int32_t data[100];
    int32_t * pdata = data;
    int32_t * odata = pdata + 50;
    for (size_t it=0; it<100; ++it) { data[it] = it + 5000; }
    std::cout << "data[10]: " << data[10] << std::endl;
    std::cout << "pdata[10]: " << pdata[10] << std::endl;
    std::cout << "*(data+20): " << *(data+20) << std::endl;
    std::cout << "*(pdata+20): " << *(pdata+20) << std::endl;
    std::cout << "data[50]: " << data[50] << std::endl;
    std::cout << "odata[0]: " << odata[0] << std::endl;
    std::cout << "data[40]: " << data[40] << std::endl;
    std::cout << "odata[-10]: " << odata[-10] << std::endl;
    std::cout << "*(data+40): " << *(data+40) << std::endl;
    std::cout << "*(odata-10): " << *(odata-10) << std::endl;
    return 0;
}
```

In [ ]:
!g++ arrays.cpp -o arrays -Wall -Wextra -Werror
!./arrays

# Floating-point

x86 architecture follows the IEEE 754-1985 standard for floating-point.  A floating-point value uses 3 fields to represent: sign, exponent (biased) (denoted by $p$), and fraction (denoted by $f$ and $f<1$).  The formula is:

$$
\pm(1+f)_2 \times 2^p
$$

Note that the number is binary-based.

x86 follows IEEE 754 standard for floating-point.  There are two commonly used floating-point formats: single and double precision.  The C++ type names are `float` and `double`, respectively.

## Single-precision (`float`)

Single-precision floating-point value uses 32 bits (4 bytes).  The first 23 bits are fraction.  The following 8 bits are exponent.  The last (highest) bit is sign; 0 is positive while 1 is negative.  In C++, the type name is `float`.

Conisder a decimal number 2.75, which we use as an example to show how to get the fields.  Write it by using the base of 2:

\begin{align*}
%
2.75 &= 2 + \frac{1}{2} + \frac{1}{2^2} = 1\times2^1 + 0\times2^0 + 1\times2^{-1} + 1\times2^{-2} \\
&= (10.11)_2 = (1.011)_2 \times 2^1 .
%
\end{align*}

The bit fields for its IEEE 754 single-precision floating-point are:

| sign (1 bit)   | exponent (8 bits)    | fraction (23 bits)             |
|----------------|----------------------|--------------------------------|
| `0`            | `1000 0000`          | `011 0000 0000 0000 0000 0000` |

The exponent bias for single-precision floating-point is 127 ($(\mathtt{0111 \, 1111})_2$).

The floating-point value is usually inexact.  For example, `0.3`, although it is rational, cannot be exactly represented as a single-precision floating-point.  Because the single-precision is 2-based, you should not follow the arithmetic intuition learned from the 10-based number system.

In [ ]:
!g++ float.cpp -o float
!./float

$(3)_{10} = (1.1)_2\times2^1$

## Double-precision (`double`)

Double-precision floating-point value uses 64 bits (8 bytes).  The first 52 bits are fraction.  The following 11 bits are exponent.  The last (highest) bit is sign; 0 is positive while 1 is negative.  In C++, the type name is `double`.

Use the same example of 2.75 for the double-precision floating-point.  Write $2.75 = (1.011)_2 \times 2^1$.  The exponent bias for double-precision floating-point is 1023 ($(\mathtt{11 \, 1111 \, 1111})_2$).  The bit fields are:

| sign (1 bit)   | exponent (11 bits)   | fraction (52 bits)                                                 |
|----------------|----------------------|--------------------------------------------------------------------|
| `0`            | `100 0000 0000`      | `0110 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000` |

Compared with the single-precision version:

| sign (1 bit)   | exponent (8 bits)    | fraction (23 bits)             |
|----------------|----------------------|--------------------------------|
| `0`            | `1000 0000`          | `011 0000 0000 0000 0000 0000` |

## Numeric limits

Both C and C++ provides constants for the value limit of each type.  In C++, the constants are available through include file `limits`.

```cpp
#include <iostream>
#include <cstdint>
#include <limits>
int main(int, char **)
{
    std::cout << "type\t\tlowest()\tmin()\t\tmax()\t\tepsilon()" << std::endl << std::endl;
    std::cout << "float\t\t"
              << std::numeric_limits<float>::lowest() << "\t"
              << std::numeric_limits<float>::min() << "\t"
              << std::numeric_limits<float>::max() << "\t"
              << std::numeric_limits<float>::epsilon() << "\t"
              << std::endl;
    std::cout << "double\t\t"
              << std::numeric_limits<double>::lowest() << "\t"
              << std::numeric_limits<double>::min() << "\t"
              << std::numeric_limits<double>::max() << "\t"
              << std::numeric_limits<double>::epsilon() << "\t"
              << std::endl;
    std::cout << "int32_t\t\t"
              << std::numeric_limits<int32_t>::lowest() << "\t"
              << std::numeric_limits<int32_t>::min() << "\t"
              << std::numeric_limits<int32_t>::max() << "\t"
              << std::numeric_limits<int32_t>::epsilon() << "\t"
              << std::endl;
    std::cout << "uint32_t\t"
              << std::numeric_limits<uint32_t>::lowest() << "\t\t"
              << std::numeric_limits<uint32_t>::min() << "\t\t"
              << std::numeric_limits<uint32_t>::max() << "\t"
              << std::numeric_limits<uint32_t>::epsilon() << "\t"
              << std::endl;
    std::cout << "int64_t\t\t"
              << std::numeric_limits<int64_t>::lowest() << "\t"
              << std::numeric_limits<int64_t>::min() << "\t"
              << std::numeric_limits<int64_t>::max() << "\t"
              << std::numeric_limits<int64_t>::epsilon() << "\t"
              << std::endl;
    std::cout << "uint64_t\t"
              << std::numeric_limits<uint64_t>::lowest() << "\t\t"
              << std::numeric_limits<uint64_t>::min() << "\t\t"
              << std::numeric_limits<uint64_t>::max() << "\t"
              << std::numeric_limits<uint64_t>::epsilon() << "\t"
              << std::endl;
    return 0;
}
```

In [ ]:
!g++ nlimits.cpp -o nlimits
!./nlimits

## Exception handling

The pragma "`#pragma STDC FENV_ACCESS ON`" turns on floating-point exception handling in CPU.  C++ defines the following floating-point exception that is supported by the hardware:

| macro | math error condition | description |
| -- | -- | -- |
| FE_DIVBYZERO | pole error | math result was infinite or undefined |
| FE_INEXACT | inexact result | rounding was required for the operation |
| FE_INVALID | domain error | the argument was outside the domain in which the math operation |
| FE_OVERFLOW | range error | the result was too large to be representable |
| FE_UNDERFLOW | range error | the result became subnormal due to loss of precision |
| FE_ALL_EXCEPT | -- | bitwise OR of all supported floating-point exceptions |

`fpexc.cpp`:

```cpp
#include <iostream>
#include <cfenv>
#include <cmath>
#include <limits>
int main(int, char **)
{
    float v1;

    feclearexcept(FE_ALL_EXCEPT);
    v1 = 0.3;
    std::cout << "result: " << v1/0 << std::endl;
    if (fetestexcept(FE_DIVBYZERO)) { std::cout << "  FE_DIVBYZERO" << std::endl; }

    feclearexcept(FE_ALL_EXCEPT);
    v1 = 2;
    std::cout << "std::sqrt(2): " << std::sqrt(v1) << std::endl;
    if (fetestexcept(FE_INEXACT)) { std::cout << "  FE_INEXACT" << std::endl; }

    feclearexcept(FE_ALL_EXCEPT);
    v1 = 2;
    std::cout << "std::acos(2): " << std::acos(v1) << std::endl;
    if (fetestexcept(FE_INVALID)) { std::cout << "  FE_INVALID" << std::endl; }

    feclearexcept(FE_ALL_EXCEPT);
    v1 = std::numeric_limits<float>::max();
    std::cout << "std::numeric_limits<float>::max() * 2: " << v1 * 2 << std::endl;
    if (fetestexcept(FE_OVERFLOW)) { std::cout << "  FE_OVERFLOW" << std::endl; }

    feclearexcept(FE_ALL_EXCEPT);
    v1 = std::numeric_limits<float>::min();
    std::cout << "std::numeric_limits<float>::min() / 10: " << v1 / 10 << std::endl;
    if (fetestexcept(FE_UNDERFLOW)) { std::cout << "  FE_UNDERFLOW" << std::endl; }

    return 0;
}
```

In [ ]:
!g++ fpexc.cpp -o fpexc
!./fpexc

# Object-oriented programming

#Object-oriented programming (OOP) allows us to organize data with logic.  The organized entities are called objects.  The point of using OOP is to make it easier to process the data.  It usually may result in fewer lines of code and make more helper functions memorizable.

## Class

A class defines how objects organize the data and logic.  C++ provides two keywords to define a class: `class` and `struct`.  In most cases one can be used to replace the other.  By default, the accessibility of `class` is `private`, if no access specifier is used.  On the other hand, `struct`'s default accessibility is `public`.

In addition to the mostly equivalent behavior, conventionally, `struct` has a strong implication that the type is a POD (plain old data).  As such, when you need a class, prefer `class` over `struct`.  If you want a POD, use `struct`.

## Encapsulation

Class is the basic unit for encapsulation, i.e., separating the interface from the implementation detail.  Users of the class should not know how the class is implemented internally.  C++ class has 3 access controls to help encapsulation.  The `private` access means only the class itself may access the member (data or functions).  The `public` access means everything can access the member.  The `protected` applies to inherited classes.  Only the defining class and its derived classes can access `protected`.

Encapsulation is very useful for numerical code.  In the first impression, the access control prevents "straight-forward" code to access data.  However, when we start development it's impossible to foresee all the logic and constructs.  Without proper encapsulation, we may not productively move forward.

## Class example

For private member data, we want a convention to distinguish them from other variables.  Prefixing `m_` is a common one.  Other popular choices include `mMember` (prefixing `m` with camel-case) and `member_` (postfixing `_`).

See the differences between `class` and `struct`.  Use `class` to define a point:

```cpp
class PointClass
{
    float m_x, m_y; // by default private.
public:
    // Accessors
    float getX() const { return m_x; }
    float getY() const { return m_y; }
    void setX(float v) { m_x = v; }
    void setY(float v) { m_y = v; }
}; /* end class PointClass */
```

Alternately, use `struct`:

```cpp
struct PointStruct
{
    float m_x, m_y; // by default public.
}; /* end class PointStruct */
```

In [ ]:
!g++ class.cpp --std=c++11 -o class
!./class

## Accessors

To access the private member data, accessors are usually needed.  It may be argued that in a good OO design, classes should not let their consumers know about its member data.  But it is impractical to eliminate data classes from numerical code.  If you cannot totally hide it from outside, accessors must be provided.

There are two major styles for accessors: get/set and one-method-name.  The second style uses a reference to directly access member data.

```cpp
class PointClass
{
    float m_x, m_y; // by default private.
public:
    // Accessors: get/set style.
    float getX() const { return m_x; }
    float getY() const { return m_y; }
    void setX(float v) { m_x = v; }
    void setY(float v) { m_y = v; }
    // Accessors of alternate style: single method name.
    float const & x() const { return m_x; } // getter
    float       & x()       { return m_x; } // setter
    float const & y() const { return m_y; } // getter
    float       & y()       { return m_y; } // setter
}; /* end class PointClass */
```

In [ ]:
!g++ accessor.cpp --std=c++11 -o accessor
!./accessor

## Reference

Reference is an important construct in C++ and used everywhere.

Like a pointer, a C++ reference allow aliasing.  But unlike a pointer, a reference cannot be constructed without initialization, and is safer than a pointer.

```cpp
int v = 10;
std::cout << "v = " << v << std::endl;
int * pv; // pointer; danger: uninitialized
pv = &v;
*pv = 11;
std::cout << "v = " << v << std::endl;
int & rv = v; // reference
rv = 12;
std::cout << "v = " << v << std::endl;
// error: declaration of reference variable 'nrv' requires an initializer
//int & nrv;
```

A const refenrece is oftern used to alias a read-only object.

```cpp
int const & crv = v; // const reference
// error: cannot assign to variable 'crv' with const-qualified type 'const int &'
//crv = 12;
```

In [ ]:
!g++ reference.cpp --std=c++11 -o reference
!./reference

# Constructor and destructor

A constructor of a class is called when an object of the class is _instantiated_ (the _object_ can also be called an _instance_).  You may have as many constructors as you like.  There are special constructors that the compiler provides even if you don't.  The most essential one is the _default constructor_.  It does not have any argument, and is called when you declare a variable of the class.

```cpp
class Line // assume it possesses heavy resources
{
public:
    Line(); // default constructor.
};
```

The other two special constructors are _copy constructor_ and _move constructor_.

```cpp
Line & Line(Line const & ); // copy constructor
Line & Line(Line       &&); // move constructor
```

A copy constructor is called when the compiler needs to copy the object:

```cpp
Line line1; // invokes default constructor
Line line2(line1); // invokes copy constructor
```

A move constructor is called when the compiler knows the object to be instantiated will move the resources from the argument:

```cpp
Line line3(std::move(line2)); // invokes move constructor
```

(Move semantics will be discussed in later lectures.)

## Destructor

When an object is no longer in use, the compiler calls its destructor to remove it from memory.  The destructor is responsible for releasing the resources that it no longer needs.  (Failure to release the unused resource is called resource leak.)

```cpp
class Line
{
public:
    Line(size_t size) : m_size(size), m_coord(new float[size*2]) {}
    // Desctructor.
    ~Line() { if (nullptr != m_coord) { delete[] m_coord; } }
private:
    size_t m_size = 0; // number of points.
    float * m_coord = nullptr; // memory buffer for the points.
};
```

## See how the constructor and destructor work

In [ ]:
!g++ constructor.cpp --std=c++11 -o constructor
!./constructor

## Copy and move assignment operators

When copy and move constructors are available, we usually also define the companion assignment operators:

```cpp
Line & operator=(Line const & ); // copy assignment operator.
Line & operator=(Line       &&); // move assignment operator.
```

They are called in the following occasions:

```cpp
Line line1, line2, line3;
line2 = line1; // copy assignment.
line3 = std::move(line2); // move assignment.
```

The assignment operators do things similar to constructors with one major difference.  The assignment operators usually check for self assignment, which should do nothing.

# Polymorphism

In C++, when a class has any member function that is virtual, it is polymorphic.  C++ compiler knows the object is polymorphic, and uses the type information to find the associated member function in runtime.

To make the class `Line` polymorphic, we add an additional virtual function `length()`, and make the destructor virtual too.

```cpp
class Line
{
public:
    Line() = default;
    Line(Line const & );
    Line(Line       &&);
    Line & operator=(Line const & );
    Line & operator=(Line       &&);
    Line(size_t size) : m_size(size), m_coord(new float[size*2]) {}
    virtual ~Line() { if (nullptr != m_coord) { delete[] m_coord; } }
    virtual float length() const;
    size_t size() const { return m_size; }
    float & x(size_t it) const { check_range(it); return m_coord[it*2  ]; }
    float & x(size_t it)       { check_range(it); return m_coord[it*2  ]; }
    float & y(size_t it) const { check_range(it); return m_coord[it*2+1]; }
    float & y(size_t it)       { check_range(it); return m_coord[it*2+1]; }
private:
    void check_range(size_t it) const
    { if (it >= m_size) { throw std::out_of_range("Line index out of range"); } }
    size_t m_size = 0; // number of points.
    float * m_coord = nullptr; // memory buffer for the points.
}; /* end class Line */
```

Then derive the class `WeighedLine` from it and override the virtual functions:

```cpp
class WeighedLine : public Line
{
public:
    WeighedLine(WeighedLine const & );
    WeighedLine(WeighedLine       &&);
    WeighedLine & operator=(WeighedLine const & );
    WeighedLine & operator=(WeighedLine       &&);
    WeighedLine(size_t size) : Line(size), m_weight(new float[size-1]) {}
    virtual ~WeighedLine() override { delete[] m_weight; }
    virtual float length() const override;
    float const & weight(size_t it) const { return m_weight[it]; }
    float       & weight(size_t it)       { return m_weight[it]; }
private:
    float * m_weight = nullptr; // weight on line segments.
}; /* end class WeighedLine */
```

The complete example is in `polymorphic.cpp`.  To make the polymorphism work correctly, the derived class also needs to implement the copy and move constructors and assignment operators by taking the base class into account. 

With the example we will observe the following things:

1. When the derived class needs addition data, the memory management becomes much more complex.
2. A pointer to the base class can point to a derived object.  (Upcasting is fine.)
3. A pointer to the derived class can NOT point to a base object.  (Downcasting is forbidden by default.)
4. To downcast polymorphic objects, you need to use `dynamic_cast` to let RTTI (run-time type information) do the work.
5. RTTI makes the object know the correct virtual function to call.  Although `line3` is declared as reference to the base class (`Line &`), `WeighedLine::length()` is called.

In [ ]:
!g++ polymorphic.cpp --std=c++11 -o polymorphic
!./polymorphic

Polymorphism incurs runtime penalty.  It is OK in two scenarios:

1. Runtime work is unavoidable.  We trust the compiler does a better job than writing branching code ourselves.
2. Timing doesn't matter.  When the code isn't in the inner loop of computation, inefficiency may be negligible.

As we see, to make polymorphism work, there is a lot of code to write.  Therefore in high-performance numerical code we use polymorphism with great caution.

# Curiously recursive template pattern (CRTP)

If we want to make a class hierarchy polymorphic without the runtime overhead, CRTP helps.  Usually the word polymorphism means _dynamic_ polymorphism, as we described in the previous section.  If the classes behave as polymorphic but all the type information is determined during _compile_ time, it is called _static_ polymorphism.

Static polymorphism has two major benefits (at the cost of being limited in compile time).  First is to not have runtime overhead.  The second is to avoid the memory overhead associated with virtual functions.  RTTI needs some information to determine types in runtime.  That is usually (if not always) accomplished by a virtual function table, which at least add one more word on _every_ polymorphic object.  For very small objects, like a resource handle, which usually occupies one or two words, it's a great overhead.

This is how a CRTP hierarchy looks like.  `PointBase` is our class template base.  `CartesianPoint` and `PolarPoint` pass themselves into the base class' template argument, so `PointBase` can see and access the derived classes' function to provide a common interface.

```cpp
template <class Derived>
class PointBase
{
public:
    constexpr static const double PI = 3.14159265358979323846;
    PointBase(float v0, float v1) : m_v0(v0), m_v1(v1) {}
    float const & v0() const { return m_v0; }
    float       & v0()       { return m_v0; }
    float const & v1() const { return m_v1; }
    float       & v1()       { return m_v1; }
    float dist() const
    {
        // Prevent the derived class from working if it doesn't define dist(),
        // just like what a pure virtual function does.
        static_assert(&PointBase<Derived>::dist != &Derived::dist,
                      "derived class must define dist()");
        return derived().dist();
    }
private:
    Derived const & derived() const { return *static_cast<Derived const *>(this); }
    float m_v0, m_v1;
}; /* end class PointBase */

class CartesianPoint : public PointBase<CartesianPoint>
{
public:
    using base_type = PointBase<CartesianPoint>;
    using base_type::base_type;
    float dist() const
    {
        return std::hypot(v0(), v1());
    }
}; /* end class CartesianPoint */

class PolarPoint : public PointBase<PolarPoint>
{
public:
    using base_type = PointBase<PolarPoint>;
    using base_type::base_type;
    float dist() const
    {
        return std::abs(v0());
    }
}; /* end class PolarPoint */
```

In [ ]:
!g++ crtp.cpp --std=c++11 -o crtp
!./crtp

# Standard template library (STL)

Containers are essential to data processing.  The STL provides efficient implementation of commonly used containers.  They can be grouped in 3 categories:

1. Sequence containers: `std::vector`, `std::array`, `std::list`, `std::forward_list`, `std::deque`.
2. Associative containers: `std::map`, `std::set`, `std::multimap`, `std::multiset`.
3. Unordered associated containers: `std::unordered_map`, `std::unordered_set`, `std::unordered_multimap`, `std::unordered_multiset`.

We will make a quick recapitulation for those we always use in numerical code: `std::vector`, `std::array`, `std::list`, `std::map`, `std::set`, `std::unordered_map`, `std::unordered_set`.

# `std::vector`

`std::vector` is one of the most useful STL containers.  Whenever thinking of using one-dimensional, variable-length arrays, we should consider whether or not `std::vector` may be applicable.

In [ ]:
!g++ vector.cpp -std=c++11 -o vector
!./vector

# `std::array`

The class template `array` provides a type-safe alternate to C-style fixed-size arrays.

In [ ]:
!g++ array.cpp -std=c++11 -o array
!./array

# `std::list`

STL `list` supports constant time insertion and deletion of elements.  Unlike `vector`, iterators and references to an element in a `list` don't get invalidated by adding or removing other elements.  The price to pay, however, is the slow random access.

`std::list` is usually implemented as a doubly-linked list.

In [ ]:
!g++ list.cpp -std=c++11 -o list
!./list

# `std::map` and `std::set`

STL `map` is an ordered container for key-value pairs.  The keys are unique and don't allow duplication.  `map` is usually implemented as a red-black tree.

STL `set` is a unique key container.  Like `map`, it's usually implemented as a red-black tree.

In [ ]:
!g++ map.cpp -std=c++11 -o map
!./map

# `std::unordered_map` and `std::unordered_set`

STL `unordered_map` is also a container for key-value pairs.  While the keys are unique and don't allow duplication, they do not have order.  `unordered_map` is usually implemented using hash table.

Search, insertion, and removal of elements in an `unordered_map` have constant time complexity.  On the other hand, those in a `map` have logarithmic time complexity.  While `unordered_map` usually offers faster runtime than `map`, it tends to use more memory since red-black trees is very efficient in memory usage.

Like `set` is a valueless version of `map`, `unordered_map` also has a valueless version called `unordered_set`.  STL `unordered_set`, like `unordered_map`, is usually implemented using hash table.

In [ ]:
!g++ unordered_map.cpp -std=c++11 -o unordered_map
!./unordered_map

# Exercises

1. In `array.cpp`, what may happen if you write the following code?
```cpp
data[-1] = 0;
```
2. Given 2 single-precision floating-point values, 0.3 and -0.3.  Reinterpret (not integer to floating-point casting) their data (bits) as 32-bit unsigned integers.  What is the integer value after performing XOR of the two integers?  Change the floating-point values to 183.2 and -183.2.  What is the value after XOR again?
3. What are the member data (including types) in the `std::vector` implementation in the STL in the compiler you use?
4. Reimplement the class `Line` by using STL containers instead of raw pointers (do not copy-n-paste the following code listing):

```cpp
class Line
{
public:
    Line();
    Line(Line const & );
    Line(Line       &&);
    Line & operator=(Line const & );
    Line & operator=(Line       &&);
    Line(size_t size);
    ~Line();
    size_t size() const;
    float & x(size_t it) const;
    float & x(size_t it);
    float & y(size_t it) const;
    float & y(size_t it);
private:
    // Member data.
}; /* end class Line */
int main(int, char **)
{
    Line line(3);
    line.x(0) = 0; line.y(0) = 1;
    line.x(1) = 1; line.y(1) = 3;
    line.x(2) = 2; line.y(2) = 5;

    Line line2(line);
    line2.x(0) = 9;

    std::cout << "line: number of points = " << line.size() << std::endl;
    for (size_t it=0; it<line.size(); ++it)
    {
        std::cout << "point " << it << ":"
                  << " x = " << line.x(it)
                  << " y = " << line.y(it) << std::endl;
    }

    std::cout << "line2: number of points = " << line.size() << std::endl;
    for (size_t it=0; it<line.size(); ++it)
    {
        std::cout << "point " << it << ":"
                  << " x = " << line2.x(it)
                  << " y = " << line2.y(it) << std::endl;
    }

    return 0;
}
```